Changes:
```python
from missing_sync import build_schema
datapicker = create_datapicker(schema=build_schema, calibrate_ubx_to_harp=False)
```

https://github.com/emotional-cities/notebooks/pull/9

In [ ]:
from modules import *
from missing_sync import build_schema
%matplotlib widget
datapicker = create_datapicker(schema=build_schema, calibrate_ubx_to_harp=False)

In [ ]:
data_path  = r'Z:\Exp_4-outdoor_walk\lisbon\sourcedata\data\OE005\Lisbon_Belem_sub-OE101005_2024-05-10T111823Z'
datapicker.reset(path = data_path)
create_dataset(datapicker = datapicker)

In [ ]:
datapicker.dataset.streams.EEG


datapicker.dataset.streams.EEG.data.eegstartdate

# dir(datapicker.dataset.streams.EEG.data)


In [ ]:
# Import .set using MNE
from eeglabio.utils import export_mne_raw

export_mne_raw(datapicker.dataset.streams.EEG.data, "eeg_data.set")

### 1. Load dataset

In [ ]:
display(datapicker)

In [ ]:
path = r'C:\Users\AndréAlmeida\NeuroGEARS Ltd\Emotional Cities - Documents\Experiment Control and Acquisition\Datasets\Lisbon\Graca\Lisbon_Graça_sub-OE105017_2024-05-08T161023Z'
datapicker.reset(path = path)
create_dataset(datapicker = datapicker)

### 2. Explore Dataset Maps

In [ ]:
exploremap(datapicker.geodata, column='tk_airquality_iaqindex_value')

### 3. Explore Dataset Streams

In [ ]:
datapicker.dataset.streams.Empatica.data.E4_Gsr


In [ ]:
plot_traces({
    'accelX': datapicker.dataset.streams.Accelerometer.data["Accl_X"],
    'temp (C)': datapicker.dataset.streams.TK.AirQuality.Temperature.data/100,
    'angleX': datapicker.dataset.streams.Accelerometer.data["Orientation_X"],
    'altitude': datapicker.dataset.georeference.elevation,
    'iaq': datapicker.dataset.streams.TK.AirQuality.IAQIndex.data,
    'Ibi': datapicker.dataset.streams.Empatica.data.E4_Hr.Value,
})

### 4 Export Dataset

#### 4.1 Export Dataset to Empatica analysis  

In [ ]:
lsl_markers = datapicker.dataset.streams.EEG.server_lsl_marker[datapicker.dataset.streams.EEG.server_lsl_marker.MarkerIdx>35000]

In [ ]:
from pluma.preprocessing.ecg import heartrate_from_ecg
ecg_hr,_ = heartrate_from_ecg(datapicker.dataset.streams.BioData.ECG,
                                         fs = 250, max_heartrate_bpm = 250.0,
                                        peak_height = 800, smooth_win = 10)
ecg_hr.plot()

In [ ]:
lsl_markers.to_csv(datapicker.selected_path+r'\lsl_markers.csv')
ecg_hr.to_csv(datapicker.selected_path+r'\ecg_hr.csv')
e4_gsr = datapicker.dataset.streams.Empatica.data.E4_Gsr.to_csv(datapicker.selected_path+r'\e4_gsr.csv')
e4_temp = datapicker.dataset.streams.Empatica.data.E4_Temperature.to_csv(datapicker.selected_path+r'\e4_temp.csv')
e4_ibi = datapicker.dataset.streams.Empatica.data.E4_Ibi.to_csv(datapicker.selected_path+r'\e4_ibi.csv')
e4_bvp = datapicker.dataset.streams.Empatica.data.E4_Bvp.to_csv(datapicker.selected_path+r'\e4_bvp.csv')
e4_acc = datapicker.dataset.streams.Empatica.data.E4_Acc.to_csv(datapicker.selected_path+r'\e4_acc.csv')
e4_hr = datapicker.dataset.streams.Empatica.data.E4_Hr.to_csv(datapicker.selected_path+r'\e4_hr.csv')
ecg = datapicker.dataset.streams.BioData.ECG.data.Value0.to_frame().to_csv(datapicker.selected_path+r'\ecg.csv')


#### 4.2 Export Dataset to OGC API

In [ ]:
record = DatasetRecord(datapicker.dataset, datapicker.geodata, properties=RecordProperties(
    title='<City> Outdoor Walk: <CityRegion> Subject <ID>',
    description='Outdoor walk data collection',
    license='CC BY-NC 4.0',
    tool='Bonsai',
    keywords=['<City>', 'Outdoor', 'Walk', 'Microclimate', 'Biosignals'],
    contacts=[
        Contact(
            name='Your Name',
            institution='Your Institution',
            email='youremail@yourdomain.com'
        )
    ],
    themes=[]
))

In [ ]:
rpath = Path(record.id)
export_geoframe_to_geojson(datapicker.geodata, rpath.with_suffix('.geojson'))
with open(rpath.with_suffix('.json'), 'w') as f:
    f.write(record.to_json())